In [ ]:
from mc_openapi.doml_mc.intermediate_model.metamodel import parse_metamodel, parse_inverse_associations
import yaml
with open("../assets/doml_meta_v2.0.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)
inv_assoc = parse_inverse_associations(mmdoc)

In [ ]:
import prettyprinter as pp
from prettyprinter.prettyprinter import IMPLICIT_MODULES
pp.install_extras(include=['dataclasses'])

In [ ]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.metamodel')
pp.pprint(mm["application_SoftwareComponent"])

### Load the DOML document here:

In [ ]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_doml_model
from mc_openapi.doml_mc import DOMLVersion
doml_document_path = "../../tests/doml/nginx-openstack_v2.0.domlx"
# doml_document_path = "../../tests/doml/nginx-openstack_v2.0_wrong_vm_iface.domlx"
with open(doml_document_path, "rb") as xmif:
    doc = xmif.read()

im, _ = parse_doml_model(doc, DOMLVersion.V2_0)
print(im)

In [ ]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_xmi_model
doml_xmi = parse_xmi_model(doc, DOMLVersion.V2_0)

In [ ]:
print(dir(doml_xmi.eClass))

In [ ]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.doml_element')
pp.pprint(im)

In [ ]:
unbound_elems_n = 0
unbound_elems = [f"unbound{i}" for i in range(unbound_elems_n)]

In [ ]:
from z3 import Solver

from mc_openapi.doml_mc.z3encoding.metamodel_encoding import (
    def_association_rel,
    def_attribute_rel,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from mc_openapi.doml_mc.z3encoding.im_encoding import (
    assert_im_associations_q,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict
)
from mc_openapi.doml_mc.z3encoding.utils import mk_adata_sort

solver = Solver()

class_sort, class_ = mk_class_sort_dict(mm, solver.ctx)
assoc_sort, assoc = mk_association_sort_dict(mm, solver.ctx)
attr_sort, attr = mk_attribute_sort_dict(mm, solver.ctx)
elem_sort, elem = mk_elem_sort_dict(im, solver.ctx, unbound_elems)
ss_sort, ss = mk_stringsym_sort_dict(im, mm, solver.ctx)
AData = mk_adata_sort(ss_sort, solver.ctx)
elem_class_f = def_elem_class_f_and_assert_classes(
    im,
    solver,
    elem_sort,
    elem,
    class_sort,
    class_
)
attr_rel = def_attribute_rel(
    attr_sort,
    elem_sort,
    AData
)
assert_im_attributes(
    attr_rel,
    solver,
    im,
    mm,
    elem,
    attr_sort,
    attr,
    AData,
    ss
)
assoc_rel = def_association_rel(
    assoc_sort,
    elem_sort
)
assert_im_associations_q(
    assoc_rel,
    solver,
    {k: v for k, v in im.items() if k not in unbound_elems},
    elem,
    assoc_sort,
    assoc,
)

In [ ]:
# solver.push()

### Metamodel statistics

In [ ]:
from mc_openapi.doml_mc.intermediate_model.metamodel import get_subclasses_dict

print("Number of classes:", len(mm))
print(
    "Number of tuples in the subclass relation:",
    sum(len(s) for s in get_subclasses_dict(mm).values())
)
n_attrs = sum(len(c.attributes) for c in mm.values())
print("Number of attributes:", n_attrs)
print("Number of attribute assertions:", 3 * n_attrs)
n_assocs = sum(len(c.associations) for c in mm.values())
print("Number of associations:", n_assocs)
print("Number of associations assertions:", 3 * n_assocs + len(inv_assoc))



### DOML model statistics

In [ ]:
n_elems = len(im)
print("Number of elements:", n_elems)
n_attrs_im = sum(len(e.attributes) for e in im.values())
print("Number of attributes (IM):", n_attrs_im)
n_assocs_im = sum(len(a) for e in im.values() for a in e.associations.values())
print("Number of associations (IM):", n_assocs_im)
print("Number of string symbols:", len(ss))
print("Number of IM attribute assertions:", n_elems)
print("Number of IM association assertions:", n_elems ** 2)

In [ ]:
solver.check()

In [ ]:
solver.unsat_core()

In [ ]:
solver.statistics()

In [ ]:
from z3 import Consts, ForAll, Exists, Implies, And, Or, Not

In [ ]:
# All VMs have a network interface.
vm, iface = Consts(
    "vm iface", elem_sort
)
vmIfaceAssertion = ForAll(
    [vm],
    Implies(
        elem_class_f(vm) == class_["infrastructure_VirtualMachine"],
        Exists(
            [iface],
            And(
                assoc_rel(vm, assoc["infrastructure_ComputingNode::ifaces"], iface)
            )
        )
    )
)
solver.assert_and_track(vmIfaceAssertion, "vm_iface")

In [ ]:
# Some VM has no network interface. 
vm, iface = Consts("vm iface", elem_sort)
vmIfaceNegAssertion = And(
    elem_class_f(vm) == class_["infrastructure_VirtualMachine"],
    Not(
        Exists(
            [iface],
            assoc_rel(vm, assoc["infrastructure_ComputingNode::ifaces"], iface)
        )
    )
)
solver.assert_and_track(vmIfaceNegAssertion, "vm_iface_neg")

In [ ]:
# nginx has a source code property
softwareComponent, prop = Consts("softwareComponent prop", elem_sort)
nginxSourceCode = ForAll(
    [softwareComponent],
    Implies(
        And(
            elem_class_f(softwareComponent) == class_["application_SoftwareComponent"],
            attr_rel(softwareComponent, attr["commons_DOMLElement::name"], AData.ss(ss["nginx"]))
        ),
        Exists(
            [prop],
            And(
                elem_class_f(prop) == class_["commons_SProperty"],
                attr_rel(prop, attr["commons_Property::key"], AData.ss(ss["source_code"])),
                assoc_rel(softwareComponent, assoc["commons_DOMLElement::annotations"], prop)
            )
        )
    )
)
solver.assert_and_track(nginxSourceCode, "nginx_source_code")

In [ ]:
solver.check()

In [ ]:
solver.statistics().memory

In [ ]:
model = solver.model()
# print(model)
print(model[vm])

In [ ]:
from itertools import product
for (e1n, e1), a, (e2n, e2) in product(elem.items(), assoc.values(), elem.items()):
    if (e1n in unbound_elems or e2n in unbound_elems) and m.eval(assoc_rel(e1, a, e2)):
        print(e1, a, e2)